# The Battle of the Neighborhoods
### Applied Data Science Capstone Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results](#results)
* [Discussion](#discussion)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

This notebook will try to find similar neighborhoods in two different cities based on the type of venues and popular spots contained in the neighborhoods. Specifically, we will look at neighborhoods in two major cities; (Manhattan) New York, New York USA and Toronto, Ontario Canada.

This project would be of interest to people moving from Toronto proper to Manhattan and are looking to settle in similar areas. This project would also be of interest to companies looking to move headquarters or open up branch offices in Manhattan in areas that are similar to the ones they currently operate in in Toronto.

## Data <a name="data"></a>

In order to complete our project we will need the following data:

* Coordinate Data for the city centers of Manhattan and Toronto, available from **GeoPy Nominatim**
* Coordinate Data for the neighborhoods in Manhattan, available at this URL: **https://cocl.us/new_york_dataset**
* Neighborhoods in Toronto by postal code, available on **Wikipedia: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M**
* Coordinate Data for postal codes in Toronto, available at this URL: **http://cocl.us/Geospatial_data**
* Coordinate Data and Venue/spot type from **Foursquare API**

Using the Foursquare API data we will classify the various neighborhoods of Manhattan based on the most common types of venues in that neighborhood.
Then, we will do the same with the various neighborhoods of Toronto proper. Finally, we will compare the neighborhoods to determine which are 'closest' to one another by examining a matrix of the relative frequencies of each type of venue.

### Libraries
Let's start by importing all the various libraries necessary to complete our project.

In [2]:
import numpy as np 

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

import folium
!conda install lxml -y

from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier

!pip install yellowbrick
from yellowbrick.cluster import KElbowVisualizer

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         238 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

### Manhattan Data
In the following code will download the Manhattan neighborhood data and save this into a Pandas dataframe.
Then we will use **Nominatim** and **Folium** to generate a map of Manhattan with the neighborhoods' locations marked.
Finally, we will make calls to the **Foursquare API** to generate a table containing venue data.

In [ ]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
nyc = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyc = nyc.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [93]:
nyc.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [94]:
manhattan = nyc[nyc['Borough'] == 'Manhattan'].reset_index(drop = True)

In [95]:
manhattan.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [96]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Manhattan are 40.7896239, -73.9598939.


In [97]:
# create map of NYC using latitude and longitude values
map_nyc = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan['Latitude'], manhattan['Longitude'], manhattan['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)  
    
map_nyc

In [137]:
CLIENT_ID = 'L'
CLIENT_SECRET = 'R'
VERSION = '20180605' 
radius = 500
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L
CLIENT_SECRET:R


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
manhattan_venues = getNearbyVenues(names=manhattan['Neighborhood'],
                                   latitudes=manhattan['Latitude'],
                                   longitudes=manhattan['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [14]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [15]:
print('There are {} unique categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 340 unique categories.


### Toronto Data
In the following code will download the Toronto neighborhood data from **Wikipedia** and save this into a Pandas dataframe along with coordinate data from this URL **http://cocl.us/Geospatial_data**.
Then we will use **Nominatim** and **Folium** to generate a map of Toronto with the neighborhoods' locations marked.
Finally, we will make calls to the **Foursquare API** to generate a table containing venue data.

In [16]:
# read the table off the webpage
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(link)

In [17]:
# first item in the list is the table needed
type(tables)
df = tables[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [18]:
# drop entries with an unassigned borough
df = df[df['Borough']!= 'Not assigned'].reset_index(drop = True)

In [19]:
# assign borough name to neighborhood name for single entry
df[df['Neighbourhood'] == 'Not assigned']
df.iloc[6][2] = df.iloc[6][1]

In [20]:
# clean up column names
df.rename(columns = {'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace = True)
df.sort_values('PostalCode', inplace = True, ascending = True)
df = df.reset_index(drop = True)

In [21]:
toronto = df[df['Borough'].str.contains("Toronto")].reset_index(drop = True)

In [22]:
# grouping by postcode and borough put neighborhoods together
toronto = toronto.groupby(['PostalCode','Borough'], as_index = False, sort = False).agg(", ".join)

In [23]:
# download coordinates
!wget -q -O 'latlon.csv' http://cocl.us/Geospatial_data

latlon_df = pd.read_csv('latlon.csv')

latlon_df.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)

# merge dataframes to obtain final dataframe
toronto = pd.merge(toronto, latlon_df, how = 'inner', on = 'PostalCode')


In [24]:
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [25]:
# use geolocator to get coordinates for Toronto
t_address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
t_location = geolocator.geocode(t_address)
t_latitude = t_location.latitude
t_longitude = t_location.longitude
print('The geograpical coordinates of Toronto, ON are {}, {}.'.format(t_latitude, t_longitude))

The geograpical coordinates of Toronto, ON are 43.655115, -79.380219.


In [26]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[t_latitude, t_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}; {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [27]:
toronto_venues = getNearbyVenues(names = toronto['Neighborhood'],
                                   latitudes = toronto['Latitude'],
                                   longitudes = toronto['Longitude']
                                  )


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
South Hill, Rathnelly, Summerhill West, Deer Park, Forest Hill SE
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Regent Park, Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
King, Adelaide, Richmond
Toronto Islands, Harbourfront East, Union Station
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, Yorkville, North Midtown
University of Toronto, Harbord
Kensington Market, Grange Park, Chinatown
Railway Lands, South Niagara, Harbourfront West, Island airport, Bathurst Quay, CN Tower, King and Spadina
Stn A PO Boxes 25 The Esplanade
Underground city, First Canadian Place
Christie
Dovercourt Village, Dufferin
Trinity, Little Portugal
Exhibition Place, Brockton, Parkdale Village
High Park, The 

In [28]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [29]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 unique categories.


We can see that there are more unique venue categories in Manhattan than in Toronto so we will have to drop any venues with venue categories not shared by both cities.

In [30]:
manhattan_venues_shared = manhattan_venues[manhattan_venues['Venue Category'].isin(toronto_venues['Venue Category'].unique())].reset_index(drop = True)

In [31]:
toronto_venues_shared = toronto_venues[toronto_venues['Venue Category'].isin(manhattan_venues['Venue Category'].unique())].reset_index(drop = True)

In [32]:
print('There are {} unique categories.'.format(len(toronto_venues_shared['Venue Category'].unique())))

There are 187 unique categories.


In [33]:
print('There are {} unique categories.'.format(len(manhattan_venues_shared['Venue Category'].unique())))

There are 187 unique categories.


## Methodology <a name="methodology"></a>

Now that we have our dataframes containing venue data from each city let's use one hot encoding to count the number of venues of each venue category in each neighborhood. 

In [34]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues_shared[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot = pd.get_dummies(toronto_venues_shared[['Venue Category']], prefix="", prefix_sep="")

In [35]:
# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues_shared['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [36]:
manhattan_onehot.shape

(2916, 188)

In [37]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_shared['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [38]:
toronto_onehot.shape

(1607, 188)

Now, we will calculate the mean of each venue category to the total number of venues in each neighborhood. This will tell us the relative frequency of that venue category to the total venues in that neighborhood. A value of 1.0 would represent that all venues in that neighborhood are of that category. A value of 0 would indicate that no venues of that category exist in that neighborhood. 
Let's define how similar two neighborhoods are by how closely their column values match each other.

In [39]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Battery Park City,0.000000,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.00000,0.000000,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.02381,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.023810,0.011905,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.011905,0.000000,0.000000,0.023810,0.000000,0.083333,0.000000,0.000000,0.011905,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.011905,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,0.000000,0.000000,0.000000,0.011905,0.011905,0.000000,0.023810,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.059524,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,0.000000,0.011905,0.000000,0.000000,0.000000,0.011905,0.000000,0.011905,0.011905,0.011905,0.000000,0.000000,0.000000,0.00000,0.011905,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.107143,0.011905,0.000000,0.011905,0.000000,0.023810,0.023810,0.023810,0.000000,0.011905,0.000000,0.000000,0.00000

In [40]:
# calculate mean number of each type of venue
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.019608,0.000000,0.019608,0.039216,0.000000,0.000000,0.00000,0.039216,0.000000,0.019608,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.039216,0.000000,0.000000,0.000000,0.039216,0.000000,0.000000,0.000000,0.019608,0.058824,0.078431,0.019608,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.019608,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.000000,0.039216,0.000000,0.00000,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.00000,0.000000,0.000000,0.000000,0.019608,0.039216,0.000000,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.019608,0.

Now let's take a look at the 10 most common venue types in each neighborhood.

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
manhattan_sorted = pd.DataFrame(columns=columns)
manhattan_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    manhattan_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

manhattan_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Italian Restaurant,Food Court,Cupcake Shop,Clothing Store,Pizza Place,Playground
1,Carnegie Hill,Coffee Shop,Pizza Place,Cosmetics Shop,Café,Yoga Studio,Bakery,Grocery Store,French Restaurant,Japanese Restaurant,Bookstore
2,Central Harlem,Chinese Restaurant,Cosmetics Shop,American Restaurant,Gym / Fitness Center,Seafood Restaurant,Bar,French Restaurant,Boutique,Event Space,Spa
3,Chelsea,Coffee Shop,Bakery,Italian Restaurant,Ice Cream Shop,Nightclub,Hotel,American Restaurant,Theater,Seafood Restaurant,Men's Store
4,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,American Restaurant,Vietnamese Restaurant,Salon / Barbershop,Optical Shop,Spa,Bubble Tea Shop,Korean Restaurant


In [43]:
# create a new dataframe
toronto_sorted = pd.DataFrame(columns=columns)
toronto_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Italian Restaurant,Farmers Market,Beer Bar,Steakhouse,Seafood Restaurant,Café,Bakery
1,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Burrito Place,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Park,Restaurant,Skate Park,Smoke Shop
2,"Cabbagetown, St. James Town",Coffee Shop,Park,Pizza Place,Café,Market,Pub,Italian Restaurant,Bakery,Restaurant,Grocery Store
3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Middle Eastern Restaurant,Sandwich Place,Ice Cream Shop,Chinese Restaurant,Bar,Sushi Restaurant
4,Christie,Café,Grocery Store,Park,Restaurant,Diner,Nightclub,Coffee Shop,Baby Store,Convenience Store,Italian Restaurant


In [44]:
manhattan_merged = manhattan

# merge manhattan_grouped with manhattan to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(manhattan_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,Coffee Shop,Sandwich Place,Yoga Studio,Supplement Shop,Gym,Ice Cream Shop,Miscellaneous Shop,Donut Shop,Discount Store,Diner
1,Manhattan,Chinatown,40.715618,-73.994279,Chinese Restaurant,Cocktail Bar,Bakery,American Restaurant,Vietnamese Restaurant,Salon / Barbershop,Optical Shop,Spa,Bubble Tea Shop,Korean Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,Café,Grocery Store,Bakery,Mexican Restaurant,Deli / Bodega,Supermarket,New American Restaurant,Chinese Restaurant,Park,Sandwich Place
3,Manhattan,Inwood,40.867684,-73.921210,Mexican Restaurant,Café,Lounge,Pizza Place,Wine Bar,American Restaurant,Caribbean Restaurant,Chinese Restaurant,Bakery,Park
4,Manhattan,Hamilton Heights,40.823604,-73.949688,Pizza Place,Café,Mexican Restaurant,Park,Coffee Shop,Indian Restaurant,Sushi Restaurant,Cocktail Bar,Bakery,Yoga Studio
5,Manhattan,Manhattanville,40.816934,-73.957385,Coffee Shop,Mexican Restaurant,Italian Restaurant,Seafood Restaurant,Park,Gastropub,Farmers Market,Ramen Restaurant,Juice Bar,Cuban Restaurant
6,Manhattan,Central Harlem,40.815976,-73.943211,Chinese Restaurant,Cosmetics Shop,American Restaurant,Gym / Fitness Center,Seafood Restaurant,Bar,French Restaurant,Boutique,Event Space,Spa
7,Manhattan,East Harlem,40.792249,-73.944182,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,Convenience Store,Sandwich Place,Restaurant,Clothing Store,Cocktail Bar
8,Manhattan,Upper East Side,40.775639,-73.960508,Italian Restaurant,Bakery,Art Gallery,Coffee Shop,Juice Bar,Gym / Fitness Center,French Restaurant,Hotel,Spa,Grocery Store
9,Manhattan,Yorkville,40.775930,-73.947118,Coffee Shop,Gym,Italian Restaurant,Bar,Pizza Place,Sushi Restaurant,Deli / Bodega,Diner,Mexican Restaurant,Japanese Restaurant


In [45]:
manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

In [46]:
X = manhattan_grouped_clustering.values
X.shape

(40, 187)

In [47]:
y = manhattan_grouped_clustering.index.values
y

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39])

In [49]:
Z = toronto_grouped_clustering.values
Z.shape

Since we are only concerned about the 'closest' neighborhood we only need to set k = 1 in the k-Nearest Neighbors algorithm.
Also, since the dimension of each element of our matrix is 187, we will simplify the calculation by only calculating the absolute difference of each column value to find the most similar Manhattan neighborhood to our Toronto neighborhoods.

In [50]:
k = 1
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k, weights = 'distance', p = 1).fit(X,y)
neigh

Now, that we've trained our model on the Manhattan neighborhoods let's use it to assign the most similar Manhattan neighborhood to each Toronto neighborhood.

In [51]:
yhat = neigh.predict(Z)
yhat

array([19, 28, 13, 26, 14, 36, 26, 39, 28, 37, 37, 30, 15,  8, 26, 30, 14,
       10, 30, 19, 39, 30, 33, 39, 16, 28,  3, 26, 21, 25, 34, 28, 31, 26,
        9,  8,  9, 19])

In [52]:
assigned_neigh = []
for n in range(yhat.shape[0]):
    assigned_neigh.append(manhattan_grouped.iloc[yhat[n]][0])

Now, let's calculate how similar a Toronto neighborhood is to the assigned Manhattan neighborhood. Looking at the Manhattan_grouped and Toronto_grouped dataframes, we can see that the values range from 0 to 1. These values represent the percentage of the total venues in each neighborhood that is of the given venue type. Thus we can see that two neighborhoods would be completely dissimilar if by subtracting their column values and adding the absolute values of these we get a value of 2. Alternatively, if doing the same process yields a value of 0, the two neighborhoods are as similar as possible within our definition of similarity. Thus, we can see that by subtracting two neighborhoods column values and adding the absolute values will obtain a value that ranges from 0 to 2. Let's subtract this value from 2 and then divide the result by 2 to obtain a percentage of how similar the neighborhoods are.

In [53]:
similarity = []
for n in range(yhat.shape[0]):
    similarity.append((2 -np.sum(np.abs(toronto_grouped_clustering.iloc[n][:]-manhattan_grouped_clustering.iloc[yhat[n]][:])))/2)

Now, we will assign the most similar Manhattan neighborhood to each Toronto neighborhood and merge this with the other Toronto data.

In [54]:
# add assigned manhattan neighborhood
toronto_sorted.insert(0, '% Similar', similarity)
toronto_sorted.insert(0, 'Manhattan Neighborhood', assigned_neigh)

In [55]:
toronto_merged = toronto

# merge toronto_grouped with toronto to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_sorted.set_index('Neighborhood'), on='Neighborhood')

Let's make sure that these choices are indeed the most similar neighborhoods as we've defined them by checking them against random choices.

In [56]:
import random

In [91]:
for n in range(yhat.shape[0]):
    print(np.sum(np.abs(toronto_grouped_clustering.iloc[n][:]-manhattan_grouped_clustering.iloc[yhat[n]][:])),np.sum(np.abs(toronto_grouped_clustering.iloc[n][:]-manhattan_grouped_clustering.iloc[random.randint(0,39)][:])))

1.2501960784313726 1.3529411764705883
1.595959595959596 1.8444444444444446
1.0921052631578947 1.4345238095238093
1.1122789028454243 1.2680722891566265
1.4999999999999998 1.5342465753424657
1.1777777777777776 1.4366013071895425
0.9205384281372123 1.2317616701853156
1.0241008527994067 1.249266862170088
1.4444444444444442 1.8181818181818181
1.5055555555555555 1.6739130434782608
1.3728070175438596 1.6629213483146068
1.6666666666666665 1.976190476190476
1.3191489361702127 1.401844532279315
1.0719640179910046 1.2739130434782608
1.0124113475177303 1.2819444444444446
1.6666666666666665 2.0
1.90625 1.9574468085106378
1.2306046013911183 1.586206896551724
1.3333333333333333 1.9166666666666667
1.2791836734693878 1.3716759431045147
1.5310344827586206 1.7857142857142856
1.3333333333333335 1.9770114942528736
1.945205479452055 2.0000000000000004
1.140886699507389 1.8285714285714285
1.0793491864831037 1.4410058027079304
1.4444444444444442 1.75609756097561
1.1034322820037106 1.2249334516415262
1.1276595

As we can see, the numbers on the right are bounded from below, i.e. they never fall less than, by the numbers on the left. Therefore, these are indeed are best choices.

In [115]:
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Manhattan Neighborhood,% Similar,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Turtle Bay,0.035714,Health Food Store,Pub,Trail,Yoga Studio,Eastern European Restaurant,Flea Market,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Sutton Place,0.417077,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Yoga Studio,Bubble Tea Shop,Café,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,Roosevelt Island,0.388889,Park,Sandwich Place,Italian Restaurant,Pet Store,Pizza Place,Pub,Movie Theater,Fast Food Restaurant,Burrito Place,Liquor Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,Manhattanville,0.443850,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Chinese Restaurant,Latin American Restaurant,Seafood Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,Stuyvesant Town,0.166667,Bus Line,Park,Yoga Studio,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,Roosevelt Island,0.277778,Clothing Store,Hotel,Park,Breakfast Spot,Gym,Food & Drink Shop,Sandwich Place,Ethiopian Restaurant,Flea Market,Fish Market
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,Flatiron,0.384698,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Furniture / Home Store,Dessert Shop,Diner,Rental Car Location,Restaurant,Chinese Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,Yorkville,0.487950,Dessert Shop,Sandwich Place,Sushi Restaurant,Gym,Pizza Place,Coffee Shop,Café,Italian Restaurant,Pharmacy,Deli / Bodega
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,Hudson Yards,0.046875,Restaurant,Yoga Studio,Electronics Store,Flower Shop,Flea Market,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
9,M4V,Central Toronto,"South Hill, Rathnelly, Summerhill West, Deer P...",43.686412,-79.400049,Roosevelt Island,0.277778,Coffee Shop,Pub,Pizza Place,Liquor Store,Fried Chicken Joint,Sports Bar,Restaurant,Supermarket,Bagel Shop,Sushi Restaurant


Now, let's map the Toronto neighborhoods along with their most similar Manhattan neighborhood and % similar in parenthesis. 

In [101]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[t_latitude, t_longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood, m_neighborhood, d in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Manhattan Neighborhood'], toronto_merged['% Similar']):
    label = '{}; {} ({}: {}%)'.format(neighborhood, borough, m_neighborhood,round(d,2)*100)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [136]:
toronto_merged[['Neighborhood','Manhattan Neighborhood', '% Similar']].sort_values(by = '% Similar', ascending = False)

,Neighborhood,Manhattan Neighborhood,% Similar
21,"Commerce Court, Victoria Hotel",Murray Hill,0.539731
20,"Toronto Dominion Centre, Design Exchange",Murray Hill,0.511735
18,"King, Adelaide, Richmond",Murray Hill,0.493794
29,"Underground city, First Canadian Place",Financial District,0.492903
7,Davisville,Yorkville,0.487950
19,"Toronto Islands, Harbourfront East, Union Station",Financial District,0.466117
26,"Kensington Market, Grange Park, Chinatown",East Village,0.464018
14,"Ryerson, Garden District",Lenox Hill,0.460325
11,"Cabbagetown, St. James Town",Hamilton Heights,0.453947
15,St. James Town,Chelsea,0.448284


## Results <a name="results"></a>

Let's now compare the Toronto neighborhood (*Commerce Court, Victoria Hotel*) with the most similarity to it's assigned Manhattan neighborhood (*Murray Hill*) to see if they share similar types and relative amounts of venues.

In [61]:
toronto_merged[toronto_merged["% Similar"] == toronto_merged["% Similar"].max()]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Manhattan Neighborhood,% Similar,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Murray Hill,0.539731,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Deli / Bodega,Seafood Restaurant,Steakhouse,Italian Restaurant


In [62]:
manhattan_merged[manhattan_merged['Neighborhood'] == 'Murray Hill']

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Manhattan,Murray Hill,40.748303,-73.978332,Coffee Shop,Hotel,Sandwich Place,Japanese Restaurant,Italian Restaurant,Gym,Gym / Fitness Center,French Restaurant,American Restaurant,Bagel Shop


As we can see, their most common venues are both **Coffee Shops**, **11%** of the total venues in *Commerce Court, Victoria Hotel* and **5.3%** of the total venues in *Murray Hill*. They also both seem to have a decent number of **Hotels**, **6%** of the total venues in *Commerce Court, Victoria Hotel* and **4%** of the total venues in *Murray Hill*. Within their top most common venues they also have a fair amount of **American Restaurants**, **4%** of the total venues in *Commerce Court, Victoria Hotel* and **3%** of total venues in *Murray Hill*. The final most common venue type is **Italian Restaurants** with **3%** of total venues for both *Commerce Court, Victoria Hotel* and *Murray Hill*, this is how they are most similar.

In [70]:
toronto_grouped[toronto_grouped['Neighborhood'] == 'Commerce Court, Victoria Hotel']

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
6,"Commerce Court, Victoria Hotel",0.0,0.040816,0.0,0.010204,0.0,0.0,0.010204,0.0,0.0,0.0,0.020408,0.0,0.020408,0.0,0.020408,0.0,0.0,0.0,0.010204,0.0,0.0,0.010204,0.0,0.0,0.020408,0.0,0.0,0.0,0.061224,0.0,0.0,0.0,0.0,0.010204,0.0,0.0,0.0,0.010204,0.112245,0.010204,0.0,0.0,0.0,0.010204,0.0,0.010204,0.0,0.030612,0.010204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010204,0.0,0.0,0.0,0.0,0.0,0.010204,0.010204,0.010204,0.010204,0.010204,0.0,0.0,0.0,0.0,0.030612,0.0,0.0,0.0,0.0,0.0,0.0,0.030612,0.010204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061224,0.0,0.0,0.010204,0.0,0.0,0.010204,0.030612,0.020408,0.010204,0.0,0.0,0.0,0.0,0.010204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010204,0.0,0.010204,0.0,0.0,0.0,0.0,0.0,0.0,0.010204,0.0,0.0,0.0,0.0,0.010204,0.0,0.0,0.010204,0.010204,0.0,0.0,0.0,0.05102,0.0,0.0,0.010204,0.0,0.010204,0.0,0.0,0.030612,0.0,0.010204,0.0,0.0,0.0,0.0,0.010204,0.0,0.0,0.0,0.0,0.0,0.030612,0.0,0.0,0.0,0.0,0.0,0.010204,0.0,0.0,0.010204,0.020408,0.0,0.0,0.0,0.020408,0.0,0.0,0.010204,0.0,0.0


In [71]:
manhattan_grouped[manhattan_grouped['Neighborhood'] == 'Murray Hill']

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
26,Murray Hill,0.0,0.031915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021277,0.021277,0.0,0.021277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010638,0.021277,0.0,0.0,0.0,0.010638,0.0,0.0,0.0,0.0,0.021277,0.0,0.0,0.0,0.021277,0.053191,0.0,0.0,0.0,0.0,0.0,0.021277,0.010638,0.0,0.010638,0.0,0.0,0.0,0.021277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010638,0.0,0.031915,0.010638,0.010638,0.0,0.0,0.0,0.010638,0.0,0.0,0.0,0.0,0.0,0.010638,0.031915,0.031915,0.0,0.010638,0.0,0.0,0.0,0.0,0.0,0.0,0.042553,0.0,0.0,0.0,0.0,0.0,0.0,0.031915,0.042553,0.010638,0.0,0.021277,0.010638,0.0,0.010638,0.0,0.010638,0.010638,0.0,0.0,0.010638,0.021277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010638,0.0,0.010638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010638,0.010638,0.0,0.010638,0.010638,0.021277,0.010638,0.0,0.0,0.021277,0.0,0.0,0.0,0.021277,0.042553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010638,0.010638,0.0,0.0,0.021277,0.0,0.0,0.0,0.021277,0.010638,0.0,0.0,0.0,0.010638,0.010638,0.0,0.0,0.0,0.010638,0.0,0.010638,0.010638,0.0,0.0


Now, let's look at the Toronto neighborhood (*Roselawn*) with the least similarity to it's assigned Manhattan neighborhood (*Tudor City*).

In [84]:
toronto_merged[toronto_merged["% Similar"] == toronto_merged["% Similar"].min()]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Manhattan Neighborhood,% Similar,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,Tudor City,0.027397,Garden,Yoga Studio,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [85]:
manhattan_merged[manhattan_merged['Neighborhood'] == 'Tudor City']


,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Manhattan,Tudor City,40.746917,-73.971219,Park,Café,Mexican Restaurant,Greek Restaurant,Sushi Restaurant,Deli / Bodega,Pizza Place,Thai Restaurant,Diner,Asian Restaurant


As we can see, none of the top most common venue types are shared between the two. However, this is the most similar Manhattan neighborhood by our definition. Therefore, we can conclude that this neighborhood is quite different from all Manhattan neighborhoods.

Now, let's calculate the overall similarity between Toronto proper and Manhattan by averaging the % similarity of every Toronto neighborhood with it's assigned Manhattan neighborhood.

In [99]:
np.mean(toronto_merged['% Similar'])

0.35083361859623813

In [106]:
np.mean(toronto_merged[toronto_merged['Borough'] == 'Downtown Toronto']['% Similar'])

0.42262816166614176

In [107]:
np.mean(toronto_merged[toronto_merged['Borough'] == 'Central Toronto']['% Similar'])

0.2490270227418106

In [108]:
np.mean(toronto_merged[toronto_merged['Borough'] == 'West Toronto']['% Similar'])

0.33259609538154694

In [109]:
np.mean(toronto_merged[toronto_merged['Borough'] == 'East Toronto']['% Similar'])

0.2975101639401842

With an overall similarity of about **35%** we can assume that in fact Toronto is not that similar to Manhattan overall. However, if we focus on each borough of Toronto we find that *Downtown Toronto* is overall **42%** similar. While the other boroughs are overall less similar than the total average with *Central Toronto* at **25%**, *West Toronto* at **33%** and *East Toronto* at **30%**.

## Discussion <a name="discussion"></a>

Through our analysis using Foursquare API data and by only looking at venue types that are common between Manhattan and Toronto proper, it can be concluded that Toronto is not very similar to Manhattan. In particular, Manhattan neighborhoods seem to be defined by the type of food available in them while Toronto neighborhoods seem to be far less so.
To answer the original question, can we recommend a neighborhood of Manhattan that is similar to each neighborhood of Toronto, the answer appears to be no. However, we can make some fairly good recommendations for neighborhoods for Downtown Toronto in particular.
However, there are a few ways we could improve our analysis and make a better recommendation.
We could lump categories of food, drinking establishments and the like into one category.
We could narrow the categories we look at, for example only retail establishments or only eateries based on the stakeholders needs.


## Conclusion <a name="conclusion"></a>

The purpose of this project was to find neighborhoods in Manhattan which are similar to neighborhoods in Toronto Proper. In order to give recommendations to people and companies moving from Toronto to Manhattan and looking for a similar neighborhood. We have come to the conclusion that the neighborhoods of these two cities are in fact not very similar overall but we can make the best recommendation for each neighborhood with the data provided by Foursquare.
As an extension to this project, it would be a good exercise to look at other large cities and see how well they compare to Manhattan and Toronto. Perhaps London or Paris would make a more suitable match to Toronto.